<a href="https://colab.research.google.com/github/MJ2610/Chat-with-document/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage 1> Document loading into standar format.

In [ ]:
! pip install langchain
!pip install python-dotenv

In order to get the api key, visit the openAI api platform, then next under your account/profile you'll find the api keys section. Use the best practises to secure the key.

In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

**PDF's**

For loading the pdf into the langchain.

In [ ]:
#! pip install pypdf
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("")#your desired doc location in this section
pages = loader.load()

In [ ]:
len(pages)

In [ ]:
page = pages[0]

In [ ]:
print(page.page_content[0:500])

In [ ]:
page.metadata

**Youtube**
For loading the youtube video.

In [ ]:
from langchain.document_loaders.generic import GenericLoader,  FileSystemBlobLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader


# ! pip install yt_dlp
# ! pip install pydub

In [ ]:
url="" # Desired youtibe video URL
save_dir="docs/youtube/"
loader = GenericLoader(
    #YoutubeAudioLoader([url],save_dir),  # fetch from youtube
    FileSystemBlobLoader(save_dir, glob="*.m4a"),   #fetch locally
    OpenAIWhisperParser()
)
docs = loader.load()

In [ ]:
docs[0].page_content[0:500]

**Webpage URL's**

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("") #desired webpage url for loading.

In [ ]:
docs = loader.load()

In [ ]:
print(docs[0].page_content[:500])

# Step 2> Splitting/Chunking for processing of the loaded data.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter # or CharacterTextSplitter

In [ ]:
chunk_size = 26 #Define chunk size and chunk overlap
chunk_overlap = 4

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [ ]:
# c_splitter = CharacterTextSplitter(
#     chunk_size=chunk_size,
#     chunk_overlap=chunk_overlap
# ) if CharacterTextSplitter used

In [ ]:
splits = text_splitter.split_documents(pages)

In [ ]:
len(splits)

**Step 3 : Embedding**

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

**Step 4: Vectorstores**


In [ ]:
# ! pip install chromadb
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = 'docs/chroma/' # for the purpose of storage in the vectorstorage

In [ ]:
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

**Similarity Search**

In [ ]:
question = "" # any concerned question you want to ask to the document

In [ ]:
docs = vectordb.similarity_search(question,k=3)

In [ ]:
vectordb.persist() # To save the instance for fututre purpose

With till above code you can retrive the information from your document with acquiring over 80 percent of the result.

In order to increase the effeciency of the output we would require to perform these.

**Retrieval**
Retrieval is the centerpiece of our retrieval augmented generation (RAG) flow.

In [ ]:
#1 MMR - Maximum marginal relevance strives to achieve both relevance to the query and diversity among the results.

docs_mmr = vectordb.max_marginal_relevance_search(question,k=3, fetch_k=3) # note how the result changes on variating the k value and also compare the result with the similarity search result to that of this result.

In [ ]:
docs_mmr[0].page_content[:100]

Here is the protoype implementation for this project and there are further steps which can be implemented like oputput formatting. As mentioned the output may contain certain unfiltered characters. They can be filtered using certain measures.